In [1]:
%load_ext autoreload
%autoreload 2

import gym
from deep_q_learning import *
cartpole_env = gym.make("CartPole-v1").env

Using TensorFlow backend.


In [2]:
agent = DQNAgent(env=cartpole_env)
print("Agent Created")

Agent Created


In [ ]:
agent.train_agent(episodes=1000, steps_per_episode=100, batch_size=32)
print("Training Done")

 26%|██████████▏                            | 262/1000 [09:00<32:08,  2.61s/it]

In [ ]:
agent.test_agent(episodes=3)